In [7]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image
import numpy as np

# Ensure PIL is compatible with new image types
Image.MAX_IMAGE_PIXELS = None

# Configuration
train_dir = 'data/train'
val_dir = 'data/val'
img_height, img_width = 224, 224
num_classes = 2  # Change this based on your specific number of classes


In [8]:
# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

Found 328 images belonging to 2 classes.


Found 78 images belonging to 2 classes.


In [9]:
# Load ResNet50 base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False


In [10]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)

# Save the trained model
model.save('resnet50_tiff_classifier.h5')

Epoch 1/50
11/11 [==============================] - 7s 500ms/step - loss: 0.7015 - accuracy: 0.5671 - val_loss: 0.6798 - val_accuracy: 0.5513
Epoch 2/50
11/11 [==============================] - 5s 434ms/step - loss: 0.6662 - accuracy: 0.5945 - val_loss: 0.6713 - val_accuracy: 0.5641
Epoch 3/50
11/11 [==============================] - 5s 468ms/step - loss: 0.6498 - accuracy: 0.6220 - val_loss: 0.6474 - val_accuracy: 0.5897
Epoch 4/50
11/11 [==============================] - 5s 434ms/step - loss: 0.6399 - accuracy: 0.6128 - val_loss: 0.6395 - val_accuracy: 0.5897
Epoch 5/50
11/11 [==============================] - 5s 430ms/step - loss: 0.6337 - accuracy: 0.6372 - val_loss: 0.6498 - val_accuracy: 0.5769
Epoch 6/50
11/11 [==============================] - 5s 432ms/step - loss: 0.6312 - accuracy: 0.6037 - val_loss: 0.5939 - val_accuracy: 0.6923
Epoch 7/50
11/11 [==============================] - 5s 439ms/step - loss: 0.6181 - accuracy: 0.6829 - val_loss: 0.6276 - val_accuracy: 0.6282
Epoch 